# M&M ASH
*Multivariate Bayesian variable selection via Adaptive Shrinkage, with applications to eQTL fine-mapping*

<hr>

## Background

![](figures/mnm.png)

This project is mainly aimed at [fine-mapping for eQTL discovery in multiple tissues](Modular_MNMASH.html#Motivation). From methodology point of view it introduces an Empirical Bayes approach, ie, [adaptive shrinkage](https://doi.org/10.1093/biostatistics/kxw041) for solving multivariate, multiple regression problem; hence the project code name `m&m ash`. 

We started building the m&m model in summer 2016. It is essentially a multivariate version of the variational algorithm that does [univariate regression with adaptive shrinkage method](https://github.com/stephenslab/mvash) (credit to [Wei](http://github.com/NKweiwang) and [Peter](https://pcarbo.github.io/)). We have developed a version assuming [identity covariance](https://github.com/gaow/mvarbvs/tree/master/writeup/identity_cov) and a version assuming [diagonal + low rank covariance](https://github.com/gaow/mvarbvs/tree/master/writeup/lr_diag_cov) (credit to [David](http://home.uchicago.edu/~dcgerard/home.html)), along with [an implementation](https://github.com/gaow/mnmashr). We then realized several limitations to it, from modeling and computational espects, that made us reconsider the design after some attempts to alleviate these issues ([limiting to a simpler problem](Start_Simple.html) and [combine genome-wide information efficiently](https://github.com/gaow/mvarbvs/tree/master/writeup/lr_diag_cov/dirichlet_ml)).

In early 2017 we decide to take a [modular approach](Modular_MNMASH.html#Step-one:-deconvoluted-effect-size-estimation) to the problem. Essentially it involves combing in a smart way the univariate `mr-ash` (multiple regression with adaptive shrinkage, under development) for model selection and [`mash`](http://dx.doi.org/10.1101/096552) for multivariate analysis. With the release of [GTEx V7](http://stephenslab.github.io/gtex-eqtls/writeup/GTEx7_Analysis_Plan.html) data ([imputation](http://stephenslab.github.io/gtex-eqtls/analysis/20170518_Imputation.html), [annotation + formatting](http://stephenslab.github.io/gtex-eqtls/analysis/20170515_RNASeq_Analysis.html) and a number of other [bioinformatics steps](http://stephenslab.github.io/gtex-eqtls/analysis.html)) our new method can be applied and demonstrated.

During our exploration of the modular approach we realized that the variational algorithm behind `mr-ash` has various issues, among which the most obvious are sensitivity to initialization and producing produces false positives. It is a large scale variable selection model good for prediction, but not fine-mapping. Again we got stuck.

Finally in late 2017 Matthew has came up with [a new variational algorithm](https://stephens999.github.io/misc/newVB.html), inspired by [FLASH](https://arxiv.org/abs/1802.06931), that natually result in fine-mapping results by design, with similar intuition as "conditional regression" (a simple ad hoc approach to do preliminary fine-mapping in the early days). [A connection with single effect analysis](https://stephens999.github.io/misc/newVB.ss.html) was later made: turns out updating each effect, conditional on others, is a normal-means problem we are familiar with -- in the M&M setting the `MASH` updates. We thus call the fine-mapping model SuSiE, for Sum of Single Effects.

## Univeriate analysis (variable selection and fine-mapping)

![](figures/m-green.jpg)

The first module is to perform multiple regression with adaptive shrinkage for variable selection and fine-mapping

### `mr-ash` model

This is being developed and implemented in the [`varbvs`](https://github.com/pcarbo/varbvs) package. We have created [simulation data](analysis/MR_ASH_Simulation.html) based on GTEx. [Here](analysis/MR_ASH_Example.html) is how to run `mr-ash` on these real / simulated data-sets we created. Here are some results:

- [Simulation study results](analysis/20170630_Simulation_Results.html) -- we've observed that `mr-ash` tend to over-shrink signals.
- Real-data analysis [code](http://stephenslab.github.io/gtex-eqtls/analysis/20170814_MR_ASH_V7.html) and [result](prototype/20170628_MR_ASH_Toy_Example.html) -- sensitive to initialization and false positives.

### `SSE` (SuSiE) model

The SSE model has been implemented in [`susieR`](https://github.com/stephenslab/susieR). To understand property of susie we designed [simulation study with various settings focusing on flavors of susie](details/20180515_SuSiE_Benchmark.html). We then benchmark it with other fine-mapping methods as a reassurance -- our claim is that susie and all other methods identify the same sets of potentially causal variates because they all fit the same model; but susie does it at low computational cost and naturally results in interpretable confidences sets.

## Multivariate analysis

![](figures/m-yellow.png)

For the multivariate part, we've continued to make improvements to [`mashr`](https://github.com/stephenslab/mashr) (credit to [Matthew](http://stephenslab.uchicago.edu/)) that makes it [potentially suitable](../analysis/20170615_MASHR_Benchmark.html) for the scale of the GTEx project that now has > 50 tissues and possibly multiple eQTL's per gene for ~20K genes.

## Project meetings

[Here](writeup/Meetings.html) is minutes for project meetings.